# Naloga 3: Zobje

In [83]:
import pandas as pd
import math
import numpy
from random import seed, randint
import matplotlib.pyplot as plt
import statistics
import random
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm
from scipy import stats
from math import sqrt

In [139]:
Zobje = pd.read_csv("Zobje.csv")
Zobje.head(10)

,dolzina,nacin,kolicina
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5
5,10.0,VC,0.5
6,11.2,VC,0.5
7,11.2,VC,0.5
8,5.2,VC,0.5
9,7.0,VC,0.5


### A naloga
**Preizkusite, ali dodajanje vitamina C vpliva na rast zob.**

Najprej poglejmo kako se spreminja povprečje dolžine zob z dodajanjem vitamina C.

In [86]:
#pričakovane vrednosti za neko količino
po_kolicni = Zobje.groupby('kolicina').mean()
po_kolicni

,dolzina
kolicina,
0.5,10.605
1.0,19.735
2.0,26.100


Razpredelnica nam da misliti, da dolžina zob raste z dodajanjem količine vitamina C. Naredimo še preizkus za to domnevo.

Domneva H0: dodajanje vitamina C vpliva na rast zob (dodajanje vitamina C bo povečalo pričakovano vrednost velikosti zoba: mi0.5 = mi1.0 = mi2.0)

Domneva H1: dodajanje vitamina C ne vpliva na rast zob (mi0.5 < mi1.0 in mi1.0 < mi2.0)

Naredimo preizkus hipoteze:

H0: mi0.5 = mi1.0

H1: mi0.5 < mi1.0

In [87]:
Zobje_pol = Zobje[Zobje.kolicina == 0.5]
Zobje_ena = Zobje[Zobje.kolicina == 1.0]
Zobje_dva = Zobje[Zobje.kolicina == 2.0]

In [143]:
n = len(Zobje_pol) #20
m = len(Zobje_ena) #20
u = len(Zobje_dva) #20

Iz strani https://www.statology.org/inverse-t-distribution-calculator/ dobimo C_alpha **1.6860**

In [88]:
stats.ttest_ind(a=Zobje_ena.dolzina, b=Zobje_pol.dolzina, alternative='greater')

Ttest_indResult(statistic=6.476647726589102, pvalue=6.331484806608257e-08)

T = **6.476647726589102** > C_alpha -> ničelno hipotezo zavržemo

----
H0: mi1.00 >= mi2.0

H1: mi1.0 < mi2.0

In [89]:
stats.ttest_ind(a=Zobje_dva.dolzina, b=Zobje_ena.dolzina, alternative='greater')

Ttest_indResult(statistic=4.90048431719355, pvalue=9.054142680908653e-06)

T = 4.90048431719355 > C_alpha -> ničelno hipotezo zavržemo

----
### B naloga
**Kateri način dodajanja je učinkovitejši? Preizkusite, ali je razlika statistično značilna.**

Najprej poglejmo kakšna so povprečja dolžin po količini dodajanja.

In [90]:
Zobje_pomarančni = Zobje[Zobje.nacin == 'OJ']
Zobje_pomarančni.groupby('kolicina').mean()

,dolzina
kolicina,
0.5,13.23
1.0,22.70
2.0,26.06


In [91]:
Zobje_neposredno = Zobje[Zobje.nacin == 'VC']
Zobje_neposredno.groupby('kolicina').mean()

,dolzina
kolicina,
0.5,7.98
1.0,16.77
2.0,26.14


In [107]:
m = Zobje_pomarančni.dolzina.count()
n = Zobje_neposredno.dolzina.count()

In [109]:
povprecjePO = Zobje_pomarančni.dolzina.mean()

In [108]:
povprecjeNE = Zobje_neposredno.dolzina.mean()

Iz povprečji ne moremo razbrati kateri način je učinkovitejši. Opravili bomo preizkus hipoteze s Studentovim t-testom.
Zapišimo ničelno in alternativno hipotezo.

H0: miPO < miNE  (pričakovana vrednost dodajanja s pomarančnim je manjša kot pričakovana za neposredno dodajanje)

H1: miPO >= miNE  (pričakovana vrednost dodajanja s pomarančnim je višja)

Iz kalkulatorja s strani https://www.statology.org/inverse-t-distribution-calculator/ poračunamo inverz Studentone porazdelitve pri stopnji tveganja 0.05 in m+n-2 = 58: dobimo **1.6716** (to je C_alpha)

Za stopnjo tveganja 0.01 pa dobimo: **2.3924**

Zanima nas ali je T poračunan po naših podatkih iz vzorca manjši kot C_alpha

1. prvi način: po formuli

In [122]:
S = 0
for i in Zobje_pomarančni.dolzina:
    S += (i - povprecjePO)**2
for i in Zobje_neposredno.dolzina:
    S += (i - povprecjeNE)

S = math.sqrt(S/(m + n - 2))

In [137]:
T = (povprecjePO - povprecjeNE)/((Zobje_pomarančni.dolzina.std() + Zobje_neposredno.dolzina.std())/2 * math.sqrt(m + n)/math.sqrt(m*n))
T

1.9271696780478498

2. način: z ugrajeno funkcijo

In [138]:
#t-test za enostransko ocenjevanje('greater')
#vrne t statistiko in vrednost p
stats.ttest_ind(a=Zobje_pomarančni.dolzina, b=Zobje_neposredno.dolzina, alternative='greater')

Ttest_indResult(statistic=1.91526826869527, pvalue=0.03019668561206424)

Za stopnjo tveganja 0.05 je T > C_0.5 -> zavrzemo nicelno hipotezo

Za stopnjo tveganja 0.01 je T < C_0.1 -> obdrzimo nicelno hipotezo

----
Ugotoviti moramo še ali je razlika statistično značilna.

Z vgrajeno funkcijo izračunamo verjetnost za naše podatke

In [97]:
#Če je p vrednost manj od alfa je statistično značilna

pvalue = 0.03019668561206424
alpha = 0.05

pvalue < alpha

True

Zavrnemo hipotezo H0